# مشروع تنقيب البيانات

## المحتويات
1. تحضير البيئة وتحميل المكتبات
2. تحميل ومعالجة البيانات
3. استكشاف وتحليل البيانات
4. تطبيق خوارزمية Apriori
5. تطبيق خوارزمية شجرة القرار
6. تطبيق خوارزمية Naive Bayes
7. تطبيق خوارزمية K-means
8. تقييم النتائج وعرض التوصيات

## 1. تحضير البيئة وتحميل المكتبات
نقوم بتحميل المكتبات الأساسية التي سنحتاجها في المشروع

In [ ]:
# المكتبات الأساسية للتحليل
import pandas as pd
import numpy as np

# مكتبات الرسم البياني
import matplotlib.pyplot as plt
import seaborn as sns

# مكتبات التعلم الآلي
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# إعدادات العرض
plt.style.use('seaborn')
sns.set_palette('husl')
%matplotlib inline

# تجاهل التحذيرات
import warnings
warnings.filterwarnings('ignore')

## 2. تحميل ومعالجة البيانات

In [ ]:
def load_data(file_path):
    """تحميل البيانات من الملف"""
    df = pd.read_csv(file_path, encoding='ISO-8859-1')
    print(f"تم تحميل البيانات بنجاح. الأبعاد: {df.shape}")
    return df

def clean_data(df):
    """تنظيف البيانات"""
    # حذف القيم المفقودة
    df = df.dropna()
    
    # حذف الطلبات الملغاة
    df = df[~df['InvoiceNo'].astype(str).str.contains('C')]
    
    # حذف السجلات ذات الكمية أو السعر السالب/الصفري
    df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
    
    return df

def add_features(df):
    """إضافة خصائص جديدة"""
    # تحويل التاريخ
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    
    # إضافة خصائص جديدة
    df['TotalAmount'] = df['Quantity'] * df['UnitPrice']
    df['Year'] = df['InvoiceDate'].dt.year
    df['Month'] = df['InvoiceDate'].dt.month
    df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
    df['Hour'] = df['InvoiceDate'].dt.hour
    
    return df

# تحميل وتجهيز البيانات
file_path = '../data/train.csv'
try:
    # تحميل البيانات
    df = load_data(file_path)
    
    # عرض معلومات أولية عن البيانات
    print("\nمعلومات عن البيانات:")
    print(df.info())
    
    print("\nإحصائيات وصفية:")
    print(df.describe())
    
    # تنظيف وتحضير البيانات
    df = clean_data(df)
    df = add_features(df)
    
    print("\nحجم البيانات بعد التنظيف:", df.shape)
    
except FileNotFoundError:
    print(f"خطأ: الملف غير موجود في المسار {file_path}")
except Exception as e:
    print(f"حدث خطأ: {str(e)}")

## 3. استكشاف وتحليل البيانات

In [ ]:
def explore_data(df):
    """استكشاف وتحليل البيانات"""
    # 1. تحليل المبيعات حسب الشهر
    plt.figure(figsize=(12, 6))
    monthly_sales = df.groupby('Month')['TotalAmount'].sum()
    monthly_sales.plot(kind='bar')
    plt.title('إجمالي المبيعات حسب الشهر')
    plt.xlabel('الشهر')
    plt.ylabel('إجمالي المبيعات')
    plt.show()
    
    # 2. تحليل المنتجات الأكثر مبيعاً
    plt.figure(figsize=(12, 6))
    top_products = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
    top_products.plot(kind='bar')
    plt.title('أعلى 10 منتجات مبيعاً')
    plt.xlabel('المنتج')
    plt.ylabel('الكمية المباعة')
    plt.xticks(rotation=45)
    plt.show()
    
    # 3. تحليل المبيعات حسب الدولة
    plt.figure(figsize=(12, 6))
    country_sales = df.groupby('Country')['TotalAmount'].sum().sort_values(ascending=False)
    country_sales.plot(kind='bar')
    plt.title('المبيعات حسب الدولة')
    plt.xlabel('الدولة')
    plt.ylabel('إجمالي المبيعات')
    plt.xticks(rotation=45)
    plt.show()

# تنفيذ التحليل الاستكشافي
if 'df' in locals():
    explore_data(df)
else:
    print("الرجاء تحميل البيانات أولاً")